<a href="https://colab.research.google.com/github/joelsoaresjr/python_strategies/blob/main/Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação e instalação das bibliotecas necessárias 

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 924 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import pandas as pd 
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [ ]:
import plotly.graph_objects as go

Extrair carteira teórica Ibovespa

In [ ]:
def busca_carteira_teorica(indice):
  url = 'http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice={}&idioma=pt-br'.format(indice.upper())
  return pd.read_html(url, decimal=',', thousands='.')[0][:-1]

In [ ]:
ibov = busca_carteira_teorica('ibov')
ibov.sort_values('Part. (%)', ascending=False, inplace = True)

In [ ]:
ibov.head()

,Código,Ação,Tipo,Qtde. Teórica,Part. (%)
77,VALE3,VALE,ON NM,2837320141,13.770
48,ITUB4,ITAUUNIBANCO,PN EDJ N1,4757466114,5.802
3,B3SA3,B3,ON NM,1930877944,5.167
62,PETR4,PETROBRAS,PN N2,4566457037,4.946
6,BBDC4,BRADESCO,PN EJ N1,4261649634,4.663


Definição dos ativos para compor a carteira

In [ ]:
acoes = ibov[['Código','Part. (%)']]
for i in range(len(acoes)):
    acoes['Código'][i] = acoes['Código'][i]+'.SA'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
dados = yf.download(list(acoes['Código'])[:10])['Adj Close'].dropna()

dados.drop(columns = ['PETR3.SA'], inplace = True)   #Retirada da PETR3 pois já consta PETR4

[*********************100%***********************]  10 of 10 completed


In [ ]:
dados.head()

,ABEV3.SA,B3SA3.SA,BBDC4.SA,ITUB4.SA,MGLU3.SA,PETR4.SA,SUZB3.SA,VALE3.SA,WEGE3.SA
Date,,,,,,,,,
2011-05-02,5.822875,7.406761,6.874311,10.688037,0.463173,11.337945,14.352365,26.837923,2.257116
2011-05-03,5.742587,7.260232,6.669914,10.322515,0.458950,11.214565,14.352365,26.224873,2.191181
2011-05-04,5.685434,7.291933,6.665498,10.337126,0.464018,11.157279,14.352365,25.737576,2.190004
2011-05-05,5.871863,7.298276,6.707459,10.454096,0.461202,10.782723,14.352365,26.062443,2.195890
2011-05-06,6.001143,7.532885,6.846602,10.702661,0.457824,10.668157,14.352365,26.067680,2.207664


Função de trade com momentum

In [ ]:
def momentum_acoes(df, momento = 30, custo = 0.00025):
    def momentum(df, momento):  # Calcula o momento das ações 
        
        df = df.pct_change().dropna()
        
        df_momentum = df.rolling(momento).sum().shift(1)
        
        for j, k in enumerate(df_momentum):
            df_momentum.rename(columns = {k : k+'_momentum'}, inplace = True)
        
        
        return df_momentum
    
    momentum_ibov  = momentum(dados, momento)
    merge = dados.pct_change().join(momentum_ibov).dropna()
    
    maior_1 = []
    maior_2 = []
    maior_3 = []
    index = []
    index_custo = []
    custo_ = []
    for b in range(0, len(merge), momento):
        maior = merge.iloc[b, 9:].sort_values(ascending = False)[:3]
        index_custo.append(merge.iloc[b, 9:].sort_values(ascending = False)[:3].name)
        custo_.append(custo)
        for j in range(b, b+59):
            if j >= len(merge):
                break
            else:
                maior_1.append(merge[maior.index[0][:8]].iloc[j])
                maior_2.append(merge[maior.index[1][:8]].iloc[j])
                maior_3.append(merge[maior.index[2][:8]].iloc[j])
                index.append(merge[maior.index[0][:8]].index[j])
                
    custo_df = pd.DataFrame(custo_, index = index_custo)
    
    
    dic = {'Maior_1': maior_1,
           'Maior_2': maior_2,
           'Maior_3': maior_3}
    
    retorno = pd.DataFrame(dic, index = index)
    retorno['Retorno Total'] = np.dot(retorno, [0.5, 0.3, 0.2])
    
    retorno = retorno.join(custo_df)

    retorno[0].fillna(0, inplace = True)
    retorno -= custo

    '''
    plt.figure(figsize = (16, 8))
    plt.title('Retorno estratégia momentum. Momento de ' + str(momento) + ' dias')
    plt.plot(retorno[['Maior_1', 'Maior_2', 'Maior_3']].cumsum().apply(np.exp),
                     alpha = 0.7)
    plt.plot(retorno['Retorno Total'].cumsum().apply(np.exp),
             color = 'black',
             linewidth = 2,
             label = 'Carteira')
    plt.legend(['Maior_1', 'Maior_2', 'Maior_3', 'Carteira'])
    '''

    fig = go.Figure(layout=go.Layout(
        title=go.layout.Title(text='Retorno estratégia momentum. Momento de ' + str(momento) + ' dias')))

    fig.add_trace(go.Scatter(x=teste.index, y=teste['Maior_1'].cumsum().apply(np.exp), mode='lines', name='Maior_1'))
    fig.add_trace(go.Scatter(x=teste.index, y=teste['Maior_2'].cumsum().apply(np.exp), mode='lines', name='Maior_2'))
    fig.add_trace(go.Scatter(x=teste.index, y=teste['Maior_3'].cumsum().apply(np.exp), mode='lines', name='Maior_3'))

    fig.add_trace(go.Scatter(x=teste.index, y=teste['Retorno Total'].cumsum().apply(np.exp), mode='lines', name='Retorno Total', line = {'color': 'black'}))

    fig.show()
    

    return retorno


In [ ]:
teste = momentum_acoes(dados, 60)

In [ ]:
teste.head()

,Maior_1,Maior_2,Maior_3,Retorno Total,0
2011-07-27,-0.007885,-0.00025,-0.032633,-0.010544,0.00000
2011-07-28,0.008825,-0.00025,-0.053129,-0.006288,-0.00025
2011-07-29,-0.020191,-0.00025,-0.028519,-0.015874,-0.00025
2011-08-01,0.001146,-0.00025,0.017205,0.003939,-0.00025
2011-08-02,-0.021166,-0.00025,-0.000964,-0.010851,-0.00025
